In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import gc

KeyboardInterrupt: 

In [ ]:
# Initialize the dictionary to hold paths for each animal
image_paths = {}

# Base directory for your dataset
base_path = "C:/Users/Melih/Coding/Bootcamp/Melih_Aygaz_Visual_Bootcamp/AwA2-data/Animals_with_Attributes2/JPEGImages/"

# List of animals to search for
animals = ["collie", "dolphin", "elephant", "fox", "moose", "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]

# Traverse the directory structure
for dirname, _, filenames in os.walk(base_path):
    for animal in animals:
        # Check if the current directory contains the animal's name
        if animal in dirname:
            # Initialize the list if the animal is encountered for the first time
            if animal not in image_paths:
                image_paths[animal] = []
            # Add all image paths for the current animal
            for filename in filenames:
                image_paths[animal].append(os.path.join(dirname, filename))

# Print the paths for verification
for animal, paths in image_paths.items():
    print(f"{animal}: {len(paths)} images")

In [ ]:
for animal, paths in image_paths.items():
    print(f"{animal}: {paths}")

In [ ]:
# Çıkış dizini (işlenmiş resimler için)
output_path = "C:/Users/Melih/Coding/Bootcamp/Melih_Aygaz_Visual_Bootcamp/AwA2-data/Animals_with_Attributes2/processed_images"
os.makedirs(output_path, exist_ok=True)

# Resim boyutu (model girişine göre)
image_size = (128, 128)  # Örnek: 128x128

# Dizin yapısını dolaşarak ilk 650 resmi seç
for dirname, _, filenames in os.walk(base_path):
    for animal in animals:
        if animal in dirname:
            if animal not in image_paths:
                image_paths[animal] = []
            for filename in filenames:
                image_paths[animal].append(os.path.join(dirname, filename))

# Her sınıf için yalnızca ilk 650 resmi işleyin
for animal, paths in image_paths.items():
    selected_paths = paths[:650]
    animal_output_dir = os.path.join(output_path, animal)
    os.makedirs(animal_output_dir, exist_ok=True)

    for i, image_path in enumerate(selected_paths):
        try:
            # Resmi yükle
            img = cv2.imread(image_path)

            # Yeniden boyutlandır ve normalize et
            img_resized = cv2.resize(img, image_size)
            img_normalized = img_resized / 255.0

            # Yeni dosya yolunu oluştur
            output_file = os.path.join(animal_output_dir, f"{i+1}.jpg")

            # Normalize edilmiş resmi kaydet
            cv2.imwrite(output_file, (img_normalized * 255).astype("uint8"))
        except Exception as e:
            print(f"Error processing {image_path}: {e}")


            # Belleği temizle
            del img
            del manipulated_img
            gc.collect()  # Çöp toplama işlemi başlat


print("Resimler başarıyla işlendi ve kaydedildi.")

In [ ]:
def load_images(directory, image_size):
    """
    Belirtilen dizindeki tüm görüntüleri yükler ve etiketleriyle birlikte döndürür.

    Args:
        directory (str): Görüntülerin bulunduğu ana dizin.
        image_size (tuple): Yeniden boyutlandırılacak görüntü boyutu (width, height).

    Returns:
        tuple: X (görüntülerin numpy dizisi), y (etiketlerin numpy dizisi).
    """
    X = []
    y = []
    label_map = {}  # Etiketleme için sınıf isimlerini haritalandır
    current_label = 0

    for subdir in os.listdir(directory):
        subdir_path = os.path.join(directory, subdir)

        if os.path.isdir(subdir_path):
            if subdir not in label_map:
                label_map[subdir] = current_label
                current_label += 1

            for file in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, file)
                if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    try:
                        # Görüntüyü yükle ve yeniden boyutlandır
                        img = cv2.imread(file_path)
                        img_resized = cv2.resize(img, image_size)
                        X.append(img_resized)
                        y.append(label_map[subdir])
                    except Exception as e:
                        print(f"Error loading image {file_path}: {e}")

    X = np.array(X, dtype=np.float32) / 255.0
    y = np.array(y, dtype=np.int32)
    return X, y

def manipulate_image(image):
    """
    Görüntüyü manipüle eden bir fonksiyon.
    """
    # Parlaklık artırma
    manipulated_img = cv2.convertScaleAbs(image, alpha=1.5, beta=30)
    
    # Beyaz dengesi
    wb = cv2.xphoto.createSimpleWB()
    manipulated_img = wb.balanceWhite(manipulated_img)

    # Kontrast artırma
    lab = cv2.cvtColor(manipulated_img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l = cv2.equalizeHist(l)
    lab = cv2.merge([l, a, b])
    manipulated_img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    return manipulated_img


In [ ]:
def save_manipulated_images(input_dir, output_dir, image_size):
    os.makedirs(output_dir, exist_ok=True)

    for animal in os.listdir(input_dir):
        animal_dir = os.path.join(input_dir, animal)
        if os.path.isdir(animal_dir):
            output_animal_dir = os.path.join(output_dir, animal)
            os.makedirs(output_animal_dir, exist_ok=True)

            for image_file in os.listdir(animal_dir):
                image_path = os.path.join(animal_dir, image_file)
                if image_file.endswith((".jpg", ".png")):
                    try:
                        # Resmi oku
                        img = cv2.imread(image_path)

                        # Manipüle et
                        manipulated_img = manipulate_image(img)

                        # Yeni dosya yolunu oluştur
                        output_image_path = os.path.join(output_animal_dir, image_file)

                        # Manipüle edilmiş görüntüyü kaydet
                        cv2.imwrite(output_image_path, manipulated_img)
                    except Exception as e:
                        print(f"Error processing {image_path}: {e}")
    gc.collect()

# Manipüle edilmiş görüntülerin kaydedileceği klasör
output_dir = "C:/Users/Melih/Coding/Bootcamp/Melih_Aygaz_Visual_Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images"
save_manipulated_images("C:/Users/Melih/Coding/Bootcamp/Melih_Aygaz_Visual_Bootcamp/AwA2-data/Animals_with_Attributes2/processed_images", output_dir, (128, 128))


In [ ]:
# İşlenmiş ve manipüle edilmiş verileri yükle
processed_images_dir = "C:/Users/Melih/Coding/Bootcamp/Melih_Aygaz_Visual_Bootcamp/AwA2-data/Animals_with_Attributes2/processed_images"
manipulated_images_dir = "C:/Users/Melih/Coding/Bootcamp/Melih_Aygaz_Visual_Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images"
image_size = (128, 128)

# Verileri yükle
X_processed, y_processed = load_images(processed_images_dir, image_size)
X_manipulated, y_manipulated = load_images(manipulated_images_dir, image_size)

# Verileri birleştir
X_combined = np.concatenate((X_processed, X_manipulated))
y_combined = np.concatenate((y_processed, y_manipulated))

print(f"Combined X shape: {X_combined.shape}, Combined y shape: {y_combined.shape}")


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# Veri setini böl
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

# Model tanımla
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(set(y_combined)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Modeli eğit
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Test seti üzerinde değerlendirme
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc}")


In [ ]:
# Modeli kaydet
model.save("C:/Users/Melih/Coding/Bootcamp/Melih_Aygaz_Visual_Bootcamp/AwA2-data/Animals_with_Attributes2/saved_models/combined_trained_model.h5")

In [ ]:
# Manipüle edilmiş verileri yükle
manipulated_images_dir = "C:/Users/Melih/Coding/Bootcamp/Melih_Aygaz_Visual_Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images"
image_size = (128, 128)

X_manipulated, y_manipulated = load_images(manipulated_images_dir, image_size)

In [ ]:
from tensorflow.keras.models import load_model

# Kaydedilmiş modeli yükle
model = load_model("C:/Users/Melih/Coding/Bootcamp/Melih_Aygaz_Visual_Bootcamp/AwA2-data/Animals_with_Attributes2/saved_models/combined_trained_model.h5")

In [ ]:
# Modeli manipüle edilmiş verilerle test et
test_loss, test_acc = model.evaluate(X_manipulated, y_manipulated, verbose=2)
print(f"Manipüle edilmiş verilerle test doğruluğu: {test_acc}")


In [ ]:
# Manipüle edilmiş verilerde tahmin yap
predictions = model.predict(X_manipulated)

# Her tahmin için en yüksek olasılığa sahip sınıfı seç
predicted_classes = np.argmax(predictions, axis=1)

# Doğru ve yanlış tahminleri analiz et
correct_predictions = (predicted_classes == y_manipulated).sum()
total_samples = len(y_manipulated)
print(f"Doğru tahmin edilen örnek sayısı: {correct_predictions}/{total_samples}")


In [ ]:
import matplotlib.pyplot as plt

# Yanlış tahmin edilenleri bulun
incorrect_indices = np.where(predicted_classes != y_manipulated)[0]

# Yanlış tahmin edilenlerden bazılarını görselleştir
random_indices = np.random.choice(incorrect_indices, 5, replace=False)

for idx in random_indices:
    plt.imshow(X_manipulated[idx])
    plt.title(f"Gerçek: {y_manipulated[idx]}, Tahmin: {predicted_classes[idx]}")
    plt.show()


In [ ]:
# Doğru tahmin edilenleri bulun
correct_indices = np.where(predicted_classes == y_manipulated)[0]

# Doğru tahmin edilenlerden bazılarını görselleştir
random_indices = np.random.choice(correct_indices, 5, replace=False)

for idx in random_indices:
    plt.imshow(X_manipulated[idx])
    plt.title(f"Gerçek: {y_manipulated[idx]}, Tahmin: {predicted_classes[idx]}")
    plt.show()

In [ ]:
def get_wb_images(image):
    """
    Gray World algoritmasını uygulayarak renk sabitliği yapar.
    """
    # Her renk kanalının ortalamasını hesapla
    avg_b = np.mean(image[:, :, 0])  # Blue
    avg_g = np.mean(image[:, :, 1])  # Green
    avg_r = np.mean(image[:, :, 2])  # Red
    
    # Ortalama parlaklık
    avg_gray = (avg_b + avg_g + avg_r) / 3
    
    # Kanalları normalize et
    scale_b = avg_gray / avg_b
    scale_g = avg_gray / avg_g
    scale_r = avg_gray / avg_r

    # Görüntüyü normalize et
    image[:, :, 0] = np.clip(image[:, :, 0] * scale_b, 0, 255)
    image[:, :, 1] = np.clip(image[:, :, 1] * scale_g, 0, 255)
    image[:, :, 2] = np.clip(image[:, :, 2] * scale_r, 0, 255)

    return image.astype(np.uint8)


In [ ]:
# Manipüle edilmiş resimlerin ve sonuçların dizinleri
manipulated_images_dir = "C:/Users/Melih/Coding/Bootcamp/Melih_Aygaz_Visual_Bootcamp/AwA2-data/Animals_with_Attributes2/manipulated_images"
output_dir = "C:/Users/Melih/Coding/Bootcamp/Melih_Aygaz_Visual_Bootcamp/AwA2-data/Animals_with_Attributes2/wb_images"
os.makedirs(output_dir, exist_ok=True)

# Tüm manipüle edilmiş test setini renk sabitliği ile işle ve kaydet
for root, _, files in os.walk(manipulated_images_dir):
    for file in files:
        input_image_path = os.path.join(root, file)

        # Çıkış klasör yapısını korumak için alt klasörler oluştur
        relative_path = os.path.relpath(root, manipulated_images_dir)
        output_subdir = os.path.join(output_dir, relative_path)
        os.makedirs(output_subdir, exist_ok=True)
        
        output_image_path = os.path.join(output_subdir, file)

        # Görüntüyü yükle
        image = cv2.imread(input_image_path)
        if image is not None:
            # Gray World algoritması uygula
            wb_image = get_wb_images(image)
            # Sonucu kaydet
            cv2.imwrite(output_image_path, wb_image)

print("Renk sabitliği tüm manipüle edilmiş test setine uygulandı.")

In [ ]:
def load_wb_images(wb_images_dir, image_size):
    X = []
    y = []
    for root, _, files in os.walk(wb_images_dir):
        for file in files:
            image_path = os.path.join(root, file)
            label = os.path.basename(root)  # Klasör adını etiket olarak al
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.resize(image, image_size)
                X.append(image)
                y.append(animals.index(label))  # `animals` listesine göre etiketle
    return np.array(X), np.array(y)

# Kullanım
wb_X_test, wb_y_test = load_wb_images(output_dir, (128, 128))

In [ ]:
# Normalize et
wb_X_test = wb_X_test / 255.0

# Modelle test et
test_loss, test_acc = model.evaluate(wb_X_test, wb_y_test)
print(f"White balance applied test accuracy: {test_acc}")